In [58]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2
import math
import sys
from pathlib import Path

sys.path.append('/media/data/robert/code/bids')
from BIDS import BIDS_FILE,calc_centroids
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from BIDS.sitk_utils import affine_registration_transform,apply_transform,nii_to_sitk
from BIDS import NII
from optparse import Option
from typing import Tuple

def load(path: Path, seg=False):
    moving = NII.load(path, False)
    moving.reorient_to_(("P", "I", "R"))  # .rescale_nib_((1, 1, 1))
    return nii_to_sitk(moving)

p = "/media/data/new_NAKO/NAKO/MRT/rawdata/105/sub-105013/"
moving = load(Path(p, "t1dixon", "sub-105013_acq-ax_rec-in_chunk-2_t1dixon.nii.gz"))
fixed = load(Path(p, "T2w", "sub-105013_acq-sag_chunk-LWS_sequ-31_T2w.nii.gz"))
fixed = load(Path(p,'mevibe',"sub-105013_acq-ax_rec-r2s_sequ-1_mevibe.nii.gz"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:

def register(fixed_image:sitk.Image,moving_image:sitk.Image):
    #initial_transform = sitk.Euler3DTransform()
    #initial_transform.SetTranslation(fixed_image.GetOrigin())
    initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
    #initial_transform = sitk.BSplineTransform([fixed_image,moving_image],3)
    # Make sure to give centerpoint of SetCenter(centerpoint) in world coordinates.

    print(initial_transform)
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.1)

    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings.
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.01, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    #registration_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.            
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [8,4])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1])
    #registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    final_transform = registration_method.Execute(fixed_image, moving_image)

    # Always check the reason optimization terminated.
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    print(final_transform)
    moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
    sitk.WriteImage(moving_resampled, 'out_test.nii.gz')
    #sitk.WriteTransform(final_transform, os.path.join(OUTPUT_DIR, 'RIRE_training_001_CT_2_mr_T1.tfm'))
register(fixed,moving)


itk::simple::Transform
 Euler3DTransform (0x55cc919fc960)
   RTTI typeinfo:   itk::Euler3DTransform<double>
   Reference Count: 1
   Modified Time: 1564653
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     1 0 0 
     0 1 0 
     0 0 1 
   Offset: [6.73039, 7.61719, -134.972]
   Center: [-89.3085, -10.8203, 327.472]
   Translation: [6.73039, 7.61719, -134.972]
   Inverse: 
     1 0 0 
     0 1 0 
     0 0 1 
   Singular: 0
   Euler's angles: AngleX=0 AngleY=0 AngleZ=0
   m_ComputeZYX = 0

Final metric value: -0.0767079680596186
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 14.
itk::simple::Transform
 CompositeTransform (0x55cc921e3a70)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 1578062
   Debug: Off
   Object Name: 
   Observers: 
     none
   Transforms in queue, from begin to end:
   >>>>>>>>>
   Euler3DTransform (0x55cc9150db40)
     RTTI typeinfo

In [55]:
def multires_registration(fixed_image, moving_image, initial_transform):
        
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetInterpolator(sitk.sitkBSpline)
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetOptimizerAsGradientDescent(learningRate=0.5, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [2,1]) 
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [1,0]) 
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    optimized_transform = sitk.AffineTransform(3)
    registration_method.SetMovingInitialTransform(initial_transform)   
    registration_method.SetInitialTransform(optimized_transform, inPlace=False)

    #registration_method.AddCommand(sitk.sitkStartEvent, registration_callbacks.metric_start_plot)
    #registration_method.AddCommand(sitk.sitkEndEvent, registration_callbacks.metric_end_plot)
    #registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, registration_callbacks.metric_update_multires_iterations) 
    #registration_method.AddCommand(sitk.sitkIterationEvent, lambda: registration_callbacks.metric_plot_values(registration_method))

    optimized_transform = registration_method.Execute(fixed_image, moving_image)
        
    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))
    print('Optimized Transform from Multires:')
    print(optimized_transform)

    return (optimized_transform)


fixed_image = fixed#sitk.ReadImage('D:/Registration_tests/USAF-1951 fixed.tif', sitk.sitkFloat32)
moving_image = moving#sitk.ReadImage('D:/Registration_tests/USAF-1951 moving three-quarter resolution.tif', sitk.sitkFloat32)

initial_transform = sitk.CenteredTransformInitializer(  fixed_image, 
                                                        moving_image, 
                                                        sitk.AffineTransform(3),
                                                        sitk.CenteredTransformInitializerFilter.GEOMETRY)
#sitk.Show(fixed_image, 'fixed')
#sitk.Show(moving_image, 'moving')
#sitk.Show(sitk.Resample(moving_image, fixed_image, sitk.Transform()), 'identity transform')
#sitk.Show(sitk.Resample(moving_image, fixed_image, initial_transform), 'initial affine transform')
print('Parameters: ' + str(initial_transform.GetParameters()))
print('FixedParameters: ' + str(initial_transform.GetFixedParameters()))

optimized_transform = multires_registration(fixed_image, moving_image, initial_transform)
final_transform = sitk.Transform(optimized_transform)
#final_transform.AddTransform(initial_transform)

#sitk.Show(sitk.Resample(moving_image, fixed_image, final_transform), 'final affine transform')
moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkBSpline, 0.0, moving_image.GetPixelID())
sitk.WriteImage(moving_resampled, 'out_test.nii.gz')
#interact(display_2d_images_with_alpha, alpha=(0.0,1.0,0.1), fixed = fixed(fixed_image), moving=fixed(moving_resampled))

Parameters: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, -79.71652986437829, -55.21800636296223, 759.3636476790196)
FixedParameters: (-2.8615951356217053, 52.01488136296223, -566.8636476790196)
Final metric value: -0.44935186638353375
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Convergence checker passed at iteration 85.
Optimized Transform from Multires:
itk::simple::Transform
 CompositeTransform (0x55cc915297f0)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 1544605
   Debug: Off
   Object Name: 
   Observers: 
     none
   Transforms in queue, from begin to end:
   >>>>>>>>>
   AffineTransform (0x55cc9232bd90)
     RTTI typeinfo:   itk::AffineTransform<double, 3u>
     Reference Count: 1
     Modified Time: 1544446
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       0.0739159 -0.0337006 0.0592827 
       -0.0139652 1.33607 0.02122 
       0.0852947 -0.00329175 0.962163 
     Of